<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="https://raw.githubusercontent.com/holoviz/holoviz/master/doc/_static/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 3. Big Data Visualization Using Datashader</h2></div>


## Big data processing considerations

### Parquet file format:  
People often use Parquet to store and operate on big data. Parquet is an open source, column-based binary data file format. It’s highly efficient, compressed and it supports flexible data formats. The file sizes of parquet files are a lot smaller than CSVs, it’s a lot cheaper to store parquet files in the cloud,  and it’s a lot faster to read in parquet files than CSVs. That’s why when people work with big data, they usually work with the parquet file format. 

There are some interesting features of Parquet file format. 

- Column pruning: CSV is row-major format. Parquet is column-major format, which means that for parquet files, consecutive element in a column is stored next to each other in memory. This allows us to read a single column of data or a few columns of data that we specify effectively.
```
import pandas as pd 
df = pd.read_parquet(
    'https://s3.amazonaws.com/datashader-data/nyc_taxi_wide.parq',
    columns = ['dropoff_x', 'dropoff_y']
)
```

- Predicate pushdown: this allows you to filter your data at the database level when you read in the data. This is more efficient since you do not need to load all the data. 
```
import pandas as pd 
df = pd.read_parquet(
    'https://s3.amazonaws.com/datashader-data/nyc_taxi_wide.parq',
    columns = ['dropoff_x', 'dropoff_y'],
    filters=[[('dropoff_y', '>', 4976288)]]
)
```

### Pandas vs Dask: 
- Most people are probably familiar with Pandas. Pandas is great for data processing. However, it is limited by your computer memory since whenever you use pandas, you load the entire dataframe in your memory. If you have a dataframe that’s larger than your computer memory, you are out of luck. And also, Pandas only uses a single core. It’s not designed to scale for data beyond a single memory. So when you use Pandas, you are not using all the computer powers you have if you have multiple cores. 

- Dask is a parallel computing library for Python. Dask partitions your data into multiple chunks, which allows us to work with data that’s much larger than the computer memory since you only need to load data into memory one chunk at a time. It also allows parallel computing and actually optimizes computations for multiple cores and data partitions.  If you are familiar with Pandas, It’s actually quite easy to use Dask. The syntax of Dask is almost the same as Pandas. For example, to read in a parquet file, you can simply change pd.read_parquet to dd.read_parquet where dd is imported from the dask.dataframe.

## Why is big data visualization hard?
From our understanding, there are two main obstacles to visualize big data.

- The first is speed. If you were to plot the 11 million data points from my example below using your regular Python plotting tools, it would be extremely slow and your Jupyter kernel would most likely crash.
- The second is image quality. Even if it doesn’t crash and you are willing to wait, most plotting libraries will simply keep drawing each new data point as a circle or other shape on top of each other, which will result in over-plotting. Even adding alpha transparency for overlapping points won’t always help in this situation. Imagine that you have a lot of points displaying on top of each other on an image: what you see will be a blob, and it will be very hard to extract information from this blob.

Datashader provides elegant and seemingly magic solutions to these two obstacles. 




## Big data visualization using Datashader — An example
This example comes from the NYC Taxi data example on pyviz.org. For the full example, please see https://examples.pyviz.org/nyc_taxi.

### Import and configure packages

Please note that in **Colab** you will need to `!pip install panel hvplot datashader`.

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip install panel hvplot datashader

In [ ]:
import hvplot.pandas
import holoviews as hv, pandas as pd, colorcet as cc
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

You will need to run the following to be able to see plots in **Colab**.

In [ ]:
if 'google.colab' in str(get_ipython()):
    def _render(self, **kw):
        hv.extension('bokeh')
        return hv.Store.render(self)
    hv.core.Dimensioned._repr_mimebundle_ = _render

### Read in data


For the very largest files, you will want to use a distributed processing library like Dask with Datashader, but here we have a Parquet file with “only” 11 million records, which Datashader can easily handle on a laptop using Pandas without any special computing resources. Here we’ll load in two columns representing taxi drop-off locations.



In [ ]:
df = pd.read_parquet(
    'https://s3.amazonaws.com/datashader-data/nyc_taxi_wide.parq',
    columns = ['dropoff_x', 'dropoff_y']
)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
%%time
# it's so slow and plot looks terrible when I use pandas default .plot
df.plot.scatter('dropoff_x', 'dropoff_y')

## Plotting
Here we plot our data using Datashader. It only took four lines of code and six milliseconds to plot the 11million rows of data for me using my laptop, overlaid on a map of the New York area:

In [ ]:
%%time
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
points = hv.Points(df, ['dropoff_x', 'dropoff_y'])
taxi_trips = datashade(points, cmap=cc.fire, width=900, height=480)
map_tiles * taxi_trips

Alternatively, instead of using thedatashade function, we can use hvplot with rasterize=True to apply rasterization using Datashader! So easy! I highly recommend using hvplot for your visualizations.

We can either overlay our plot on a map tile:

In [ ]:
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
plot = df.hvplot(
    'dropoff_x',
    'dropoff_y',
    kind='scatter',
    rasterize=True,
    cmap=cc.fire,
    cnorm='eq_hist'
)
map_tiles * plot

Or we can use the argument `titles='EsriImagery'` directly in our `hvplot` method: 

In [ ]:
df.hvplot(
    'dropoff_x',
    'dropoff_y',
    kind='scatter',
    rasterize=True,
    cmap=cc.fire,
    cnorm='eq_hist', 
    tiles='EsriImagery'
)

If you were running it live, you would then be able to zoom in to any region of this map, with the plot dynamically updating to use the full resolution for that zoom level.

# How does Datashader work? 


![](assets/datashader_pipeline.png)

Datashader turns your data into a plot using a five-step pipeline. The Datashader docs illustrate how the pipeline works in each of the steps — projection, aggregation, transformation, colormapping, and embedding. I’m going to break down my previous example into these small steps so that we can see exactly what Datashader is doing under the hood.

Let’s first install the underlying Datashader functions so we can run through the individual steps:


In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
from holoviews.operation.datashader import rasterize

## Projection

First, we define a 2D canvas with width and height for the data to be projected onto. The canvas defines how many pixels we would like to see in the final image, and optionally defines the x_range and y_range that will map to these pixels. Here the data ranges to plot are not set in the Canvas, so they will be filled in automatically in the next step from the max and min of the data x and y values in the dataframe. The canvas defines what the projection will be, but for speed each point is actually projected during the aggregation step.

In [ ]:
canvas = ds.Canvas(plot_width=900, plot_height=480)

In [ ]:
canvas

## Aggregation

After we define the projected canvas, we project each point into the two-dimensional output grid and aggregate the results per pixel. Datashader supports many options for such aggregation, but in this example, we simply count how many data points are projected into each pixel, by iterating through the data points and incrementing the pixel where that point lands. The result in this case is a two-dimensional histogram counting dropoffs per pixel:

In [ ]:
canvas.points(df, 'dropoff_x', 'dropoff_y', agg=ds.count())

## Transformation (optional)

The result from the previous step is now a fixed-size grid, no matter how large the original dataset was. Once the data is in this grid, we can do any kind of transformation we like on it, such as selecting only a certain range of counts, masking the data based on the result of other datasets or values, etc. Here, the dropoff data ranges from zero in some pixels to tens of thousands in others, and if we try to plot the grid directly we would see only a few hotspots. To make all the different levels visible as in the image above, the data is transformed using the image-processing technique “histogram equalization” to reveal the distribution of the counts rather than their absolute values. Histogram equalization is actually folded into the colormapping step below, but we can do explicit transformations at this stage if we want, such as squaring the counts:

In [ ]:
import numpy as np

In [ ]:
np.power(canvas.points(df, 'dropoff_x', 'dropoff_y', agg=ds.count()),2)

## Colormapping

Next, we can render the binned grid data to the corresponding pixels of an image. Each bin value is mapped into one of the 256 colors defined in a colormap, either by linear interpolation or with an automatic transformation (e.g. by calling the log function on each value, or as here using histogram equalization). Here we’re using the “fire” colormap from Colorcet, which starts at black for the lowest counts (1 and 2 dropoffs) and goes through red for higher values (in the hundreds) and then yellow for even higher values (in the thousands) and finally white for the highest counts per pixel (in the tens of thousands in this case). We set the background to black to better visualize the data.

In [ ]:
tf.set_background(
    tf.shade(
        canvas.points(df, 'dropoff_x', 'dropoff_y', agg=ds.count()),
        cmap=cc.fire
    ),
    'black'
)

## Embedding

As you can see, Datashader only renders the data, not any axes, colorbars, or similar features you’d expect in a full plot. To get those features that help you interpret the data, we can embed the images generated by Datashader into a plot. The easiest way is to use HoloViews, which is a high-level plotting API that provides the flexibility to use either Matplotlib, Bokeh, or Plotly as the backend. Here is an example of using HoloViews to define a “points” object and then datashading all the points. Here we demonstrate an alternative method `rasterize` instead `datashade` so that Bokeh is in charge of the transformation and colormapping steps and allows hover and colorbars to work.



In [ ]:
map_tiles = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
points = hv.Points(df, ['dropoff_x', 'dropoff_y'])
ropts = dict(tools=['hover'], colorbar=True, colorbar_position='bottom', cmap=cc.fire, cnorm='eq_hist')
taxi_trips = rasterize(points).opts(**ropts)
map_tiles * taxi_trips

# Why is Datashader crazy fast?
First, we need to talk about the original data format. Datashader is so fast that reading in the data is usually the slowest step, particularly if your original data is a bunch of JSON files or CSV files. The Parquet file format is usually a good choice for columnar data like the dropoff points, because it is compact, quick to load in, efficiently reads in only the columns and ranges you need, and supports distributed and out-of-core operation when appropriate.

Second, with the right input file formatting, we can investigate the next most expensive task, which is the combined projection+aggregation step. This step requires calculating values for each of the millions of data points, while all subsequent calculations use the final fixed-size grid and are thus much faster. So, what does Datashader do to make this step fast?

- Datashader’s aggregation calculations are written in Python but then just-in-time compiled into wicked-fast machine code using Numba. E.g., here is the code where the counting per bin is happening.
- The example above uses a CPU, but Datashader + Numba also supports CUDA cudf dataframes as a drop-in replacement for a Pandas dataframes that run even faster if you have a GPU.
- Datashader can also parallelize its pipeline (code example) so that you can make use of all the computing cores you have available, scaling to even larger datasets and giving even faster results.

<div class="alert alert-info">
<h4>Exercise</h4>

Try change the background tile of the plot. Hint: take a look at available tiles [here](https://holoviews.org/reference/elements/bokeh/Tiles.html) . 
        
<details><summary><i><u>(Solution)</u><i></summary><br>
    
```python

from holoviews.element.tiles import EsriStreet
map_tiles = EsriStreet().opts(alpha=0.5, width=900, height=480, bgcolor='black')
plot = df.hvplot(
    'dropoff_x',
    'dropoff_y',
    kind='scatter',
    rasterize=True,
    cmap=cc.fire,
    cnorm='eq_hist'
)
map_tiles * plot
```
or 
    
```
df.hvplot(
    'dropoff_x',
    'dropoff_y',
    kind='scatter',
    rasterize=True,
    cmap=cc.fire,
    cnorm='eq_hist',
    tiles='EsriStreet'
)    
```

</details>
</div>


<div class="alert alert-info">
<h4>Reading time</h4>

Because Datashader is so fast, we can actually visualize big data interactively, dynamically redrawing whenever we zoom or pan. Here is [an example](https://examples.pyviz.org/nyc_taxi/dashboard.html) where you can view the NYC Taxi data interactively in a Panel dashboard. My favorite example on [ship traffic](https://examples.pyviz.org/ship_traffic/ship_traffic.html) illustrates that even though all you see is a pixelated image that Datashader renders, you can still inspect individual data points and understand what you are seeing. The other examples at examples.pyviz.org and datashader.org show Datashader for much larger files, up to billions of points on an ordinary laptop.

<br>    
    
Read the [Datashader documentation](https://datashader.org) and let us know if you have any questions.
</div>